# How to add examples to the prompt

:::info Prerequisites

This guide assumes familiarity with the following:

- [Query analysis](/docs/tutorials/query_analysis)

:::

As our query analysis becomes more complex, the LLM may struggle to understand how exactly it should respond in certain scenarios. In order to improve performance here, we can add examples to the prompt to guide the LLM.

Let's take a look at how we can add examples for the LangChain YouTube video query analyzer we built in the [query analysis tutorial](/docs/tutorials/query_analysis).

## Setup

### Install dependencies

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  zod uuid
</Npm2Yarn>
```

### Set environment variables

```
# Optional, use LangSmith for best-in-class observability
LANGSMITH_API_KEY=your-api-key
LANGCHAIN_TRACING_V2=true
```

## Query schema

We'll define a query schema that we want our model to output. To make our query analysis a bit more interesting, we'll add a `subQueries` field that contains more narrow questions derived from the top level question.

In [21]:
import { z } from "zod";

const subQueriesDescription = `
If the original question contains multiple distinct sub-questions,
or if there are more generic questions that would be helpful to answer in
order to answer the original question, write a list of all relevant sub-questions.
Make sure this list is comprehensive and covers all parts of the original question.
It's ok if there's redundancy in the sub-questions, it's better to cover all the bases than to miss some.
Make sure the sub-questions are as narrowly focused as possible in order to get the most relevant results.`

const searchSchema = z.object({
    query: z.string().describe("Primary similarity search query applied to video transcripts."),
    subQueries: z.array(z.string()).optional().describe(subQueriesDescription),
    publishYear: z.number().optional().describe("Year video was published")
})

## Query generation

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />
```

In [ ]:
import { ChatPromptTemplate } from "@langchain/core/prompts"
import { RunnablePassthrough, RunnableSequence } from "@langchain/core/runnables"

const system = `You are an expert at converting user questions into database queries.
You have access to a database of tutorial videos about a software library for building LLM-powered applications.
Given a question, return a list of database queries optimized to retrieve the most relevant results.

If there are acronyms or words you are not familiar with, do not try to rephrase them.`

const prompt = ChatPromptTemplate.fromMessages(
[
    ["system", system],
    ["placeholder", "{examples}"],
    ["human", "{question}"],
]
)
const llmWithTools = llm.withStructuredOutput(searchSchema, {
  name: "Search",
})
const queryAnalyzer = RunnableSequence.from([
  {
    question: new RunnablePassthrough(),
  },
  prompt,
  llmWithTools
]);

Let's try out our query analyzer without any examples in the prompt:

In [34]:
await queryAnalyzer.invoke(
  "what's the difference between web voyager and reflection agents? do both use langgraph?"
)

{
  query: "difference between Web Voyager and Reflection Agents",
  subQueries: [ "Do Web Voyager and Reflection Agents use LangGraph?" ]
}

## Adding examples and tuning the prompt

This works pretty well, but we probably want it to decompose the question even further to separate the queries about Web Voyager and Reflection Agents.

To tune our query generation results, we can add some examples of inputs questions and gold standard output queries to our prompt.

In [35]:
const examples = []

In [36]:
const question = "What's chat langchain, is it a langchain template?"
const query = {
  query: "What is chat langchain and is it a langchain template?",
  subQueries: ["What is chat langchain", "What is a langchain template"],
}
examples.push({ "input": question, "toolCalls": [query] })

1

In [37]:
const question = "How to build multi-agent system and stream intermediate steps from it"
const query = {
  query: "How to build multi-agent system and stream intermediate steps from it",
  subQueries: [
    "How to build multi-agent system",
    "How to stream intermediate steps from multi-agent system",
    "How to stream intermediate steps",
  ],
}

examples.push({ "input": question, "toolCalls": [query] })

2

In [38]:
const question = "LangChain agents vs LangGraph?"
const query = {
  query: "What's the difference between LangChain agents and LangGraph? How do you deploy them?",
  subQueries: [
    "What are LangChain agents",
    "What is LangGraph",
    "How do you deploy LangChain agents",
    "How do you deploy LangGraph",
  ],
}
examples.push({ "input": question, "toolCalls": [query] });

3

Now we need to update our prompt template and chain so that the examples are included in each prompt. Since we're working with LLM model function-calling, we'll need to do a bit of extra structuring to send example inputs and outputs to the model. We'll create a `toolExampleToMessages` helper function to handle this for us:

In [44]:
import {
    AIMessage,
    BaseMessage,
    HumanMessage,
    SystemMessage,
    ToolMessage,
  } from "@langchain/core/messages";
  import { v4 as uuidV4 } from "uuid";
  
  const toolExampleToMessages = (example: Example | Record<string , any>): Array<BaseMessage> => {
    const messages: Array<BaseMessage> = [new HumanMessage({ content: example.input })];
    const openaiToolCalls = example.toolCalls.map((toolCall) => {
      return {
        id: uuidV4(),
        type: "function" as const,
        function: {
          name: "search",
          arguments: JSON.stringify(toolCall),
        },
      };
    });
  
    messages.push(new AIMessage({ content: "", additional_kwargs: { tool_calls: openaiToolCalls } }));
  
    const toolOutputs = "toolOutputs" in example ? example.toolOutputs : Array(openaiToolCalls.length).fill("You have correctly called this tool.");
    toolOutputs.forEach((output, index) => {
      messages.push(new ToolMessage({ content: output, tool_call_id: openaiToolCalls[index].id }));
    });
  
    return messages;
  }
  
  const exampleMessages = examples.map((ex) => toolExampleToMessages(ex)).flat();

In [40]:
import { ChatPromptTemplate, MessagesPlaceholder } from "@langchain/core/prompts";
import { RunnableSequence } from "@langchain/core/runnables";

const queryAnalyzerWithExamples = RunnableSequence.from([
  {
    question: new RunnablePassthrough(),
    examples: () => exampleMessages,
  },
  prompt,
  llmWithTools
]);

In [41]:
await queryAnalyzerWithExamples.invoke(
    "what's the difference between web voyager and reflection agents? do both use langgraph?"
)

{
  query: "Difference between Web Voyager and Reflection agents, do they both use LangGraph?",
  subQueries: [
    "Difference between Web Voyager and Reflection agents",
    "Do Web Voyager and Reflection agents use LangGraph"
  ]
}

Thanks to our examples we get a slightly more decomposed search query. With some more prompt engineering and tuning of our examples we could improve query generation even more.

You can see that the examples are passed to the model as messages in the [LangSmith trace](https://smith.langchain.com/public/102829c3-69fc-4cb7-b28b-399ae2c9c008/r).

## Next steps

You've now learned some techniques for combining few-shotting with query analysis.

Next, check out some of the other query analysis guides in this section, like [how to deal with high cardinality data](/docs/how_to/query_high_cardinality).